In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import matplotlib.dates as mdates
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
from Ags_model import runAgs, calc_LE

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')
#NOTE: df_Stor is NOT FILTERED

In [ ]:
from FilterData import Filter_GPP_LE_NEE_VPD 
df_Comb_filter2 = Filter_GPP_LE_NEE_VPD(df_Comb,fqc=None)

In [ ]:
# Run A-gs model
#TODO: DEFINITIVELY REPLACE OLD FILTER WITH NP.WHERE FILTER, I TEMP HOTFIXED IT HERE:
an_final,an_umol,rs, ra = runAgs(df_profile_filter,df_Comb_filter2,df_meteo_filter,df_EC_filter,fstr=1.0)

In [ ]:
#TODO: UPDATE THIS FUNCTION EVERYWHERE TO ACTUALLY USE FILTERED VALUES!!!!
def init_ETframe(rs_series):
    df_ET = pd.concat([df_meteo_filter['L(o)'],df_meteo_filter['Te-L(o)'],df_profile_filter['Pressure'],df_Comb_filter2['VPD'],df_Comb_filter2['rH'],df_meteo_filter['P(mast)']],axis=1,sort=False)
    #convert Pressure from hPa to kPa 
    df_ET['p_kPa']=df_ET['Pressure']/10
    df_ET['VPD_adj']=df_ET['VPD'].loc[df_ET['VPD']>0] #some outlier values for VPD are negative, remove from dataset
    df_ET['VPD_adj']=df_ET['VPD_adj']/10  # VPD from df_Comb is in hPa, I need kPa, so hPa/10 = kPa
    df_ET['rs']=rs_series.to_frame(name="rs")
    df_ET['ra']=ra.to_frame(name="ra")
    return df_ET

In [ ]:
df_ET=init_ETframe(rs)

In [ ]:
df_ET=calc_LE(df_ET)

In [ ]:
#
df_ET['ET']

In [ ]:
#import precipication KNMI
P_k = pd.read_csv('knmi_rain_daily.csv')
P_k['date']=pd.to_datetime(P_k['date'])
P_k.index=P_k.date
P_k.drop(columns=['date','days_since_rain'],inplace=True) #remove date because it's duplicate, remove days_since_rain because it wont be used here
#P_k has columns 'RD' (daily rain from 8pm to 8am next day) and range 2001-01-01 - 2020-12-31 , freq=daily
P_k_30m=P_k.resample('30min').ffill()

# calc fstr

In [ ]:
df_tmp30m['Obs_LE'].where(df_tmp30m['Obs_LE']<0)

In [ ]:
#assemble
df_tmp30m=pd.concat([df_Comb_filter2['LE'],df_ET['ET_VPD'],df_ET['P(mast)'],P_k_30m],axis=1)
###df_tmp30m['P(mast)']=df_ET['P(mast)']
###df_tmp30m=df_tmp30m.dropna()
#df_tmp30m=df_tmp30m.loc[df_tmp30m['RD']==0]

df_tmp30m.where(df_tmp30m.loc[:,'RD']==0,inplace=True) # remember to use the 30m rain or it will force a re-sample to daily instead of 30m
df_tmp30m.where(df_tmp30m.loc[:,'LE']>0,inplace=True) # cant have negative observed LE
df_tmp30m.where(df_tmp30m.loc[:,'ET_VPD']>0,inplace=True) # cant have negative simulated LE

df_tmp30m.dropna(axis='index',how='all',inplace=True)
df_tmp30m.rename(columns={'LE':'Obs_LE','ET_VPD':'Sim_LE','P(mast)':'P(mast)','RD':'P_dailysum'},inplace=True)
df_tmp30m['fstr']=df_tmp30m['Obs_LE']/df_tmp30m['Sim_LE']
df_tmp30m

In [ ]:
#clamp values so it can't be over 1.
df_tmp30m_clamp=df_tmp30m.copy()
df_tmp30m_clamp.loc[:,'fstr']=df_tmp30m.where(df_tmp30m.loc[:,'fstr']>1,1)

In [ ]:
st='2001-08-17'
ed='2002-09-30'
df_tmp30m_clamp.loc[st:ed,['fstr']].plot()

### create the 3h, 1D and 7D averages, for day, and for max

### resample to 3h, 1d, 7d

In [ ]:
# we used to calculate P_3day from Pmast but that has been removed

#from 30m values to 3h averages
df_tmp3h_fst     =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].resample('3H').mean()
df_tmp3h_P       =df_tmp30m.loc[:,['P_dailysum','P(mast)']].resample('3H').max() #NOTE! the 30m knmi rain values are ffill of the daily! so we pass max just to select one
#the above line is a hotfix and breaks the Pmast values! Pmast needs to be summed, P_dailysum needs to have first() or max() applied to it.
df_tmp3h         =pd.concat([df_tmp3h_fst,df_tmp3h_P],axis=1)
df_tmp3h['P_3day']=df_tmp3h['P_dailysum'].rolling('72H').sum() #overwrite the correct P_3day

df_tmp3h_fst_day =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('3H').mean()
df_tmp3h_P_day   =df_tmp30m.loc[:,['P_dailysum','P(mast)']].between_time("08:00", "18:00").resample('3H').max()
df_tmp3h_day     =pd.concat([df_tmp3h_fst_day,df_tmp3h_P_day],axis=1)

#df 3hour mean (c = clamped fstr to max 1.0)
df_tmp3hc_fst     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('3H').mean()
df_tmp3hc_P       =df_tmp30m_clamp.loc[:,['P_dailysum','P(mast)']].between_time("08:00", "18:00").resample('3H').max()
df_tmp3hc         =pd.concat([df_tmp3h_fst,df_tmp3h_P],axis=1)

df_tmp3hc_fst_day     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].resample('3H').mean()
df_tmp3hc_P_day       =df_tmp30m_clamp.loc[:,['P_dailysum','P(mast)']].resample('3H').max()
df_tmp3hc         =pd.concat([df_tmp3hc_fst_day,df_tmp3hc_P_day],axis=1)

#rainy days are already removed using knmi data
#dont need this anymore:
##remove rainy days through Pmast
##df_rain3h_dailymax=df_tmp3h['P(mast)'].resample('1D').sum().resample('3H').ffill()
##df_rain3h_dailymax=df_rain3h_dailymax[:-1] #remove last entry because all 3h frames end on 21:00 not 00:00
##df_rain3h_dailymax=df_rain3h_dailymax.rename('P(mast)Dmax')
##df_tmp3h['P(mast)Dmax']=df_rain3h_dailymax
##df_tmp3h=df_tmp3h.loc[df_tmp3h['P(mast)Dmax']==0]

In [ ]:
#resample to daily
df_tmp1d_fst     =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].resample('D').mean()
df_tmp1d         =pd.concat([df_tmp1d_fst,P_k['RD']],axis=1)  #note! the 30m precip values are ffill from the daily, so just sub the daily values back in
df_tmp1d.rename(columns={'RD':'P_dailysum'},inplace=True)
#df_tmp30m.loc[:,['P_dailysum']
df_tmp1d['doy']  =df_tmp1d.index.dayofyear

df_tmp1d_fst_day     =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('D').mean()
df_tmp1d_day         =pd.concat([df_tmp1d_fst_day,P_k['RD']],axis=1)
df_tmp1d_day.rename(columns={'RD':'P_dailysum'},inplace=True)
df_tmp1d_day['doy']  =df_tmp1d_day.index.dayofyear


#clamped
df_tmp1dc_fst     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].resample('D').mean()
df_tmp1dc         =pd.concat([df_tmp1dc_fst,P_k['RD']],axis=1)
df_tmp1dc.rename(columns={'RD':'P_dailysum'},inplace=True)
df_tmp1dc['doy']  =df_tmp1dc.index.dayofyear

df_tmp1dc_fst_day     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('D').mean()
df_tmp1dc_day         =pd.concat([df_tmp1dc_fst_day,P_k['RD']],axis=1)
df_tmp1dc_day.rename(columns={'RD':'P_dailysum'},inplace=True)
df_tmp1dc_day['doy']  =df_tmp1dc_day.index.dayofyear

#special case: daily max

df_tmp1dcm_fst     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].resample('D').max()
df_tmp1dcm         =pd.concat([df_tmp1dcm_fst,P_k['RD']],axis=1)
df_tmp1dcm.rename(columns={'RD':'P_dailysum'},inplace=True)
df_tmp1dcm['doy']  =df_tmp1dc.index.dayofyear

In [ ]:
#resample to weekly (7 days)

df_tmp7d_fst     =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].resample('7D').mean()
df_tmp7d_P       =df_tmp1d.loc[:,['P_dailysum']].resample('7D').sum() #NOTE! for precip time we're summing the daily values, not 30m
df_tmp7d         =pd.concat([df_tmp7d_fst,df_tmp7d_P],axis=1)
#df_tmp7d['doy']  =df_tmp7d.index.dayofyear

df_tmp7d_fst_day     =df_tmp30m.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('7D').mean()
df_tmp7d_P_day       =df_tmp1d.loc[:,['P_dailysum']].between_time("08:00", "18:00").resample('7D').sum()
df_tmp7d_day         =pd.concat([df_tmp7d_fst_day,df_tmp7d_P],axis=1)
#df_tmp7d_day['doy']  =df_tmp7d_day.index.dayofyear


#clamped
df_tmp7dc_fst     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].resample('7D').mean()
df_tmp7dc_P       =df_tmp30m_clamp.loc[:,['P_dailysum']].resample('7D').sum()
df_tmp7dc         =pd.concat([df_tmp7d_fst,df_tmp7d_P],axis=1)
#df_tmp7dc['doy']  =df_tmp7dc.index.dayofyear

df_tmp7dc_fst_day     =df_tmp30m_clamp.loc[:,['Obs_LE','Sim_LE','fstr']].between_time("08:00", "18:00").resample('7D').mean()
df_tmp7dc_P_day       =df_tmp30m_clamp.loc[:,['P_dailysum']].between_time("08:00", "18:00").resample('7D').sum()
df_tmp7dc_day         =pd.concat([df_tmp7d_fst_day,df_tmp7d_P],axis=1)
#df_tmp7dc_day['doy']  =df_tmp7dc_day.index.dayofyear



### check values

In [ ]:
st='2004-01-17'
ed='2006-12-30'
df_tmp1dcm.loc[st:ed,'fstr'].plot()

In [ ]:
st='2004-01-17'
ed='2006-12-30'
df_tmp1dc.loc[st:ed,'fstr'].plot()

### end of check values

# Export to csv

In [ ]:
# put export to csv here
df_tmp1d.to_csv('./fstrOutput/NoRain/fstr_1d.csv')
df_tmp1d_day.to_csv('./fstrOutput/NoRain/fstrMax_1d_day.csv')

df_tmp1dc.to_csv('./fstrOutput/NoRain/fstr_1d_clamped.csv')
df_tmp1dc_day.to_csv('./fstrOutput/NoRain/fstr_1d_clamped_day.csv')

df_tmp1dcm.to_csv('./fstrOutput/NoRain/fstrMax_1d_clamped_day.csv')

# Toolbox and experiments with fstr

In [ ]:
## fstr max


In [ ]:
plt.rcParams["figure.figsize"] = (20,6)

#st="2005-04-01"
#ed="2005-09-30"

def fstr_plot_wrapper(st,ed):

    fig,ax = plt.subplots()
    
    b=df_ET.loc[st:ed,['P(mast)']].resample('1D').sum()
    a=(knmi_regen.loc[st:ed,['RD']]/10)
    ax.bar(b.index,b['P(mast)'],alpha=0.5, align='edge', label="Pmast data")
    ax.bar(a.index,a['RD'],alpha=0.5, align='edge', label="KNMI data")
    
    df_tmp30m=pd.DataFrame()
    df_tmp30m['ET']=df_ET['ET_VPD']
    df_tmp30m['P(mast)']=df_ET['P(mast)']
    df_tmp30m=df_tmp30m.dropna()
    
    df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
    df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)
    c=df_tmp30m.loc[st:ed].resample('1D').sum()
    c['LE/ET']=c['LE']/c['ET']
    
    ax.set_ylabel("precip")
    ax2 = ax.twinx()
    ax2.set_ylim(0,5)
    ax2.set_ylabel("fstr")
    #ax2.set_yscale('log')
    
    ax2.scatter(c.index,c['LE/ET'],c=np.where(c['LE/ET']<1,'b','r'))
    
    ax.legend()
    ax.xaxis.set_minor_locator(mdates.DayLocator())

for year in range(2001,2021):
    st=str(year)+"-01-01"
    ed=str(year)+"-12-30"
    fstr_plot_wrapper(st,ed)

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)
st="2011-04-01"
ed="2011-09-30"
#df_tmp.loc[st:ed,'LE/ET'].plot()

def signal_plot_wrapper(data,st,ed,gridarg=None,tick_frq='Day'):
    fig,ax = plt.subplots()
    fig.subplots_adjust(right=0.75)
    
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    
    # Offset the twin axis below the host
    #fig.subplots_adjust(bottom=0.2)
    twin2.spines["right"].set_position(("axes", 1.05))
    twin1.set_ylabel("Precip 3day (Pmast) [mm]")
    twin2.set_ylabel("Precip knmi [tenth of mm]")

    if '30m' in data:
        ax.plot(df_tmp30m.loc[st:ed,'LE'],label="LE obs 30m freq")
        ax.plot(df_tmp30m.loc[st:ed,'ET'],label="LE model 30m freq")
    if '3h' in data:
        ax.plot(df_tmp3h.loc[st:ed,'LE'],label="LE obs 3H mean")
        ax.plot(df_tmp3h.loc[st:ed,'ET'],label="LE model 3H mean")
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'LE'],c='g')
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'ET'],c='r')
    if '7d' in data:
        ax.plot(df_tmp7d.loc[st:ed,'LE'],c='#1f77b4',linestyle='--',label="LE obs 7day mean")
        ax.plot(df_tmp7d.loc[st:ed,'ET'],c='orange',linestyle='--',label="LE model 7day mean")

    twin1.plot(df_ET.loc[st:ed,'P(mast)'],c='red',label="Pmast")
    twin1.plot(df_ET.loc[st:ed,'last3day_prec'],c='black',label="Pmast 72h")
    twin2.bar(knmi_regen.loc[st:ed].index,knmi_regen.loc[st:ed,'RD'],width=1, align='edge', color="blue",alpha=0.5, label="KNMI Precip")[0]
    twin2.bar(df_tmp1d.loc[st:ed].index,df_tmp1d.loc[st:ed,'P(mast)sum'],width=1, align='edge', color="red",alpha=0.5, label="Pmast Precip")[0]
    
    if tick_frq=='Day':
        ax.xaxis.set_minor_locator(mdates.DayLocator())
    elif tick_frq=='Hour':
        ax.xaxis.set_minor_locator(mdates.HourLocator())
    
    ax.set_ylabel('LE [Wm-2]')
    
    ax.legend(loc=("upper left"))
    fig.autofmt_xdate()
    if gridarg==None:
        ax.grid()
    else:
        ax.grid(which=gridarg)
    #df_tmp30.loc[st:ed,'LE'].plot(label="LE obs 3H mean",legend="y",ylabel="Wm-2",title="LEobs and LEsim 3H mean and 7day mean values, 00:00-23:30")
    #df_tmp30.loc[st:ed,'ET'].plot(label="LE model 3H mean",legend="y")
    #df_tmp.loc[st:ed,'LE'].plot(label="LE obs 7day mean",legend="y",c='#1f77b4',style='--')
    #df_tmp.loc[st:ed,'ET'].plot(label="LE model 7day mean",legend="y",c='orange',style='--')
    #df_ET.loc[st:ed,'P(mast)'].plot()
    plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)
st="2011-04-01"
ed="2011-09-30"
#df_tmp.loc[st:ed,'LE/ET'].plot()

def signal_plot_wrapper_max(data,st,ed,gridarg=None,tick_frq='Day'):
    fig,ax = plt.subplots()
    fig.subplots_adjust(right=0.75)
    
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    
    # Offset the twin axis below the host
    #fig.subplots_adjust(bottom=0.2)
    twin2.spines["right"].set_position(("axes", 1.05))
    twin1.set_ylabel("Precip 3day (Pmast) [mm]")
    twin2.set_ylabel("Precip knmi [tenth of mm]")

    data_max = data.resample('D').apply()

    if '30m' in data:
        ax.plot(df_tmp30m.loc[st:ed,'LE'],label="LE obs 30m freq")
        ax.plot(df_tmp30m.loc[st:ed,'ET'],label="LE model 30m freq")
    if '3h' in data:
        ax.plot(df_tmp3h.loc[st:ed,'LE'],label="LE obs 3H mean")
        ax.plot(df_tmp3h.loc[st:ed,'ET'],label="LE model 3H mean")
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'LE'],c='g')
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'ET'],c='r')
    if '7d' in data:
        ax.plot(df_tmp7d.loc[st:ed,'LE'],c='#1f77b4',linestyle='--',label="LE obs 7day mean")
        ax.plot(df_tmp7d.loc[st:ed,'ET'],c='orange',linestyle='--',label="LE model 7day mean")

    twin1.plot(df_ET.loc[st:ed,'P(mast)'],c='red',label="Pmast")
    twin1.plot(df_ET.loc[st:ed,'last3day_prec'],c='black',label="Pmast 72h")
    twin2.bar(knmi_regen.loc[st:ed].index,knmi_regen.loc[st:ed,'RD'],width=1, align='edge', color="blue",alpha=0.5, label="KNMI Precip")[0]
    twin2.bar(df_tmp1d.loc[st:ed].index,df_tmp1d.loc[st:ed,'P(mast)sum'],width=1, align='edge', color="red",alpha=0.5, label="Pmast Precip")[0]
    
    if tick_frq=='Day':
        ax.xaxis.set_minor_locator(mdates.DayLocator())
    elif tick_frq=='Hour':
        ax.xaxis.set_minor_locator(mdates.HourLocator())
    
    ax.set_ylabel('LE [Wm-2]')
    
    ax.legend(loc=("upper left"))
    fig.autofmt_xdate()
    if gridarg==None:
        ax.grid()
    else:
        ax.grid(which=gridarg)
    #df_tmp30.loc[st:ed,'LE'].plot(label="LE obs 3H mean",legend="y",ylabel="Wm-2",title="LEobs and LEsim 3H mean and 7day mean values, 00:00-23:30")
    #df_tmp30.loc[st:ed,'ET'].plot(label="LE model 3H mean",legend="y")
    #df_tmp.loc[st:ed,'LE'].plot(label="LE obs 7day mean",legend="y",c='#1f77b4',style='--')
    #df_tmp.loc[st:ed,'ET'].plot(label="LE model 7day mean",legend="y",c='orange',style='--')
    #df_ET.loc[st:ed,'P(mast)'].plot()
    plt.show()

In [ ]:
for year in range(2002,2003): #zoom in on only the spring
    st=str(year)+"-02-01"
    ed=str(year)+"-02-07"
    signal_plot_wrapper(['30m','3h','7d'],st,ed)

In [ ]:
test_data=df_Comb.loc['2002-01-01':'2002-01-7','LE']
test_data.loc['']

In [ ]:

data_max = data.resample('D').max()
data_max